In [1]:
!pip install git+https://github.com/alibaba/TinyNeuralNetwork.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/alibaba/TinyNeuralNetwork.git to /tmp/pip-req-build-r4rp0knm
  Running command git clone -q https://github.com/alibaba/TinyNeuralNetwork.git /tmp/pip-req-build-r4rp0knm
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2 MB 27.2 MB/s 
     |████████████████████████████████| 109 kB 70.1 MB/s 
     |████████████████████████████████| 546 kB 58.6 MB/s 
  Created wheel for TinyNeuralNetwork: filename=TinyNeuralNetwork-0.1.0.20221004231442+a521163753e4b201a4a31d3cb82e5aa1fe328d93-py3-none-any.whl size=352164 sha256=92c48cdb5d1989e566151e1c7f36509e72aae138c7402fd08ae52b399f5cbe16
  Stored in directory: /tmp/pip-ephem-wheel-cache-xf6mclbb/wheels/64/6b/f5/9cf69de054ba0de53e572bc2f13988a664f3dc9623e6f0825d
Successfully built TinyNeuralNetwork


In [2]:
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5hbm6qzm
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-5hbm6qzm
     |████████████████████████████████| 4.9 MB 31.8 MB/s 
     |████████████████████████████████| 6.6 MB 56.5 MB/s 
     |████████████████████████████████| 163 kB 54.7 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175010 sha256=f5dd91e9a6f992d3d0ba2da84bcfcd8ee48a100e6a38f8c4cad9f7d5ac90177c
  Stored in directory: /tmp/pip-ephem-wheel-cache-gbdo3skd/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper


In [3]:
!git clone https://github.com/usefulsensors/openai-whisper.git

Cloning into 'openai-whisper'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 57 (delta 8), reused 5 (delta 5), pack-reused 45
Unpacking objects: 100% (57/57), done.


## Convert from pytorch to onnx

In [5]:
import whisper
import torch

tiny_model = whisper.load_model("tiny")
torch.onnx.export(tiny_model.encoder, torch.randn(1,80,3000).to("cuda"), "/content/whisper-encoder.onnx")
torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258]]).to("cuda"), torch.randn(1,384,384).to("cuda")), "/content/whisper-decoder_main.onnx")
torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258, 50259, 50359]]).to("cuda"), torch.randn(1, 384, 384).to("cuda")), "/content/whisper-decoder_language.onnx")

####Encoder TFLite model

In [6]:
!pip install onnx-tf

from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-encoder.onnx'
tf_model_path = 'model_tf-encoder'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 226 kB 30.4 MB/s 
     |████████████████████████████████| 13.1 MB 61.2 MB/s 
     |████████████████████████████████| 1.1 MB 60.4 MB/s 


In [7]:
import tensorflow as tf

saved_model_dir = 'model_tf-encoder'
tflite_model_path = 'whisper-encoder.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

##Decoder TFLite model

In [8]:
from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-decoder_main.onnx'
tf_model_path = 'model_tf-decoder_main'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

In [9]:
import tensorflow as tf

saved_model_dir = 'model_tf-decoder_main'
tflite_model_path = 'whisper-decoder_main.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

##Decoder language TFLite model

In [10]:
from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-decoder_language.onnx'
tf_model_path = 'model_tf-decoder_language'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

In [11]:
import tensorflow as tf

saved_model_dir = 'model_tf-decoder_language'
tflite_model_path = 'whisper-decoder_language.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [12]:
%ls -la

total 484964
drwxr-xr-x 1 root root      4096 Oct  4 23:17 ./
drwxr-xr-x 1 root root      4096 Oct  4 23:09 ../
drwxr-xr-x 4 root root      4096 Sep 26 13:44 .config/
drwxr-xr-x 4 root root      4096 Oct  4 23:17 model_tf-decoder_language/
drwxr-xr-x 4 root root      4096 Oct  4 23:16 model_tf-decoder_main/
drwxr-xr-x 4 root root      4096 Oct  4 23:16 model_tf-encoder/
drwxr-xr-x 4 root root      4096 Oct  4 23:15 openai-whisper/
drwxr-xr-x 1 root root      4096 Sep 26 13:45 sample_data/
-rw-r--r-- 1 root root 197228870 Oct  4 23:15 whisper-decoder_language.onnx
-rw-r--r-- 1 root root  29584512 Oct  4 23:17 whisper-decoder_language.tflite
-rw-r--r-- 1 root root 197225670 Oct  4 23:15 whisper-decoder_main.onnx
-rw-r--r-- 1 root root  29569408 Oct  4 23:16 whisper-decoder_main.tflite
-rw-r--r-- 1 root root  32857838 Oct  4 23:15 whisper-encoder.onnx
-rw-r--r-- 1 root root  10095776 Oct  4 23:16 whisper-encoder.tflite


In [13]:
!git lfs install
!git clone https://huggingface.co/zhuzilin/whisper-openvino-tiny.en

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'whisper-openvino-tiny.en'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.
Filtering content: 100% (8/8), 367.54 MiB | 24.65 MiB/s, done.


##Encoder model from zhuzilin/whisper-openvino-tiny.en

In [14]:
from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-openvino-tiny.en/encoder.onnx'
tf_model_path = '/content/whisper-openvino-tiny.en/encoder'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)
import tensorflow as tf

saved_model_dir = '/content/whisper-openvino-tiny.en/encoder'
tflite_model_path = '/content/whisper-openvino-tiny.en/encoder.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

##Run Inference on encoder tflite file(Crashing at this time)

In [ ]:
import tensorflow as tf
import numpy as np
tflite_model_path = '/content/whisper-openvino-tiny.en/encoder.tflite'


# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

print("== Input details ==")
print("name:", interpreter.get_input_details()[0]['name'])
print("shape:", interpreter.get_input_details()[0]['shape'])
print("type:", interpreter.get_input_details()[0]['dtype'])

print("\nDUMP INPUT")
print(interpreter.get_input_details()[0])

print("\n== Output details ==")
print("name:", interpreter.get_output_details()[0]['name'])
print("shape:", interpreter.get_output_details()[0]['shape'])
print("type:", interpreter.get_output_details()[0]['dtype'])

print("\nDUMP OUTPUT")
print(interpreter.get_output_details()[0])

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model with random data
input_shape = input_details[0]['shape']

interpreter.invoke()

== Input details ==
name: serving_default_mel:0
shape: [   1   80 3000]
type: <class 'numpy.float32'>

DUMP INPUT
{'name': 'serving_default_mel:0', 'index': 0, 'shape': array([   1,   80, 3000], dtype=int32), 'shape_signature': array([   1,   80, 3000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

== Output details ==
name: PartitionedCall:0
shape: [   1 1500  384]
type: <class 'numpy.float32'>

DUMP OUTPUT
{'name': 'PartitionedCall:0', 'index': 557, 'shape': array([   1, 1500,  384], dtype=int32), 'shape_signature': array([   1, 1500,  384], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


##Decoder model from zhuzilin/whisper-openvino-tiny.en

In [15]:
from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-openvino-tiny.en/decoder.onnx'
tf_model_path = '/content/whisper-openvino-tiny.en/'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)
import tensorflow as tf

saved_model_dir = '/content/whisper-openvino-tiny.en/decoder'
tflite_model_path = '/content/whisper-openvino-tiny.en/decoder.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

BackendIsNotSupposedToImplementIt: ignored

## Convert from pytorch to tflite using Tinynn

In [ ]:
import whisper
import torch

import torch.nn as nn
from tinynn.converter import TFLiteConverter
from tinynn.graph.quantization.quantizer import PostQuantizer
from tinynn.graph.tracer import model_tracer
from tinynn.util.train_util import DLContext, get_device, train

model = whisper.load_model("tiny")
model.eval()
audio = whisper.load_audio("/content/openai-whisper/filename.mp3")
audio = whisper.pad_or_trim(audio)
print(audio.shape)
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)
print(mel.shape)
quantizer = PostQuantizer(model, mel, work_dir='out', config={'asymmetric': True, 'per_tensor': False})
#qat_model = quantizer.quantize()
